In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import os
from pprint import pp
from bittorrent import BitTorrentClient

In [3]:
# Create client
DIR = 'http_examples'
def get_random_metainfo_filepath():
    random_file = random.choice(os.listdir(DIR))
    return f'{DIR}/{random_file}'
client = BitTorrentClient(get_random_metainfo_filepath())

In [ ]:
# Test MetaInfo parsing
print("MetaInfo Dict:")
pp(client.metainfo.__dict__)
print("Announce URLS:")
pp(client.metainfo.announce_urls)

In [ ]:
# Test Tracker parsing
print("Tracker Dicts:")
pp([tracker.__dict__ for tracker in client.trackers])

In [4]:
# Prepare for download
client.prepare()
# client.peers = client._get_peers()

[{b'ip': b'72.28.130.143', b'port': 24812}, {b'ip': b'176.213.135.85', b'port': 6881}, {b'ip': b'189.29.146.186', b'port': 51413}, {b'ip': b'93.216.7.216', b'port': 39798}, {b'ip': b'5.144.66.84', b'port': 6881}, {b'ip': b'185.107.13.235', b'port': 54542}, {b'ip': b'46.114.238.62', b'port': 6980}, {b'ip': b'185.148.0.134', b'port': 30918}, {b'ip': b'95.110.181.225', b'port': 13264}, {b'ip': b'1.21.119.10', b'port': 18928}, {b'ip': b'185.159.158.107', b'port': 45794}, {b'ip': b'198.144.156.32', b'port': 1}, {b'ip': b'185.84.175.17', b'port': 53823}, {b'ip': b'46.149.125.26', b'port': 51413}, {b'ip': b'87.122.225.74', b'port': 26158}, {b'ip': b'209.121.33.215', b'port': 6881}, {b'ip': b'150.143.58.42', b'port': 51413}, {b'ip': b'92.255.207.93', b'port': 52500}, {b'ip': b'139.47.71.64', b'port': 51413}, {b'ip': b'93.141.160.78', b'port': 29909}, {b'ip': b'216.24.40.188', b'port': 60061}, {b'ip': b'45.134.213.217', b'port': 1860}, {b'ip': b'188.114.67.53', b'port': 41554}, {b'ip': b'149.28

In [ ]:
# Test Peers parsing
print("Peers:")
pp([peer.__dict__ for peer in client.peers])

In [ ]:
# Test Pieces generation
print("Pieces:")
pp([piece.__dict__ for piece in client.pieces[:50]])

In [ ]:
for peer in client.peers:
    peer.setup()

In [ ]:
import time

def get_valid_peer():
    peer = random.choice(client.peers)
    try:
        peer.connect()
        return peer
    except TimeoutError:
        return get_valid_peer()

def get_handshaked_peer():
    valid_peer = get_valid_peer()
    print(f"Peer: {valid_peer.ip}:{valid_peer.port}")
    valid_peer.send_handshake()
    time.sleep(1)
    valid_peer.receive_message()
    return valid_peer if valid_peer.handshaked else get_handshaked_peer()

valid_peer = get_handshaked_peer()

In [ ]:
valid_peer.handshaked

In [ ]:
valid_peer.send_interested()

In [ ]:
message = valid_peer.receive_message()
print(message)

In [ ]:
print("Bitfield:")
print(valid_peer.bitfield)

In [ ]:
piece = valid_peer.pending_piece
if piece:
    print(piece.request_message.__dict__)
else:
    print("No pending piece")

In [ ]:
valid_peer.receive_message()
# print(valid_peer.pending_piece) # Should be None
if piece:
    print(piece.data) # Should be filled
    print(len(piece.data), client.metainfo.piece_length) # Should be == client.metainfo.piece_length